In [154]:
import pandas as pd 
import openpyxl as pxl
import numpy as np 
import statistics as st
from scipy.optimize import minimize, Bounds, LinearConstraint
from datetime import datetime

In [169]:
df=pd.read_csv("/Users/tejasvichabbra/Desktop/Portfolio_Analysis/data/combined2.csv")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"])
df=df[df['SERIES']=='EQ']
print(df, end="\n\n")
companies=df['SYMBOL'].unique()
final_day=pd.read_csv('finalday.csv')
final_day=final_day[final_day['SERIES']=='EQ']
final_day_companies=final_day['SYMBOL'].unique()
path_first_day="/Users/tejasvichabbra/Desktop/Portfolio_Analysis/firstday_data1/firstday2.csv"
first_day=pd.read_csv(path_first_day)
first_day=first_day[first_day['SERIES']=='EQ']
first_day_companies=first_day['SYMBOL'].unique()
final_day_companies = set(final_day_companies).intersection(first_day_companies)
# print(companies)
# len(companies)
# print(len(final_day_companies))
# companies1=[]
# print(len(companies1))
# companies=companies1
# #inal_day_data=final_day.loc(:,['SYMBOL','CLOSE'])

           SYMBOL SERIES    CLOSE  TIMESTAMP
0       20MICRONS     EQ   125.60 2012-11-01
2         3MINDIA     EQ  3820.00 2012-11-01
3          A2ZMES     EQ    59.95 2012-11-01
4       AANJANEYA     EQ   637.30 2012-11-01
5      AARTIDRUGS     EQ   193.05 2012-11-01
...           ...    ...      ...        ...
89272  ZODIACLOTH     EQ   170.20 2012-09-03
89273   ZODJRDMKJ     EQ    21.30 2012-09-03
89274   ZUARIAGRO     EQ   137.10 2012-09-03
89275   ZYDUSWELL     EQ   393.70 2012-09-03
89276       ZYLOG     EQ   291.05 2012-09-03

[80717 rows x 4 columns]



In [171]:
df.sort_values(by=['SYMBOL','TIMESTAMP'],inplace=True)
companies=df['SYMBOL'].unique()
companies1=[]
for x in companies:
    for j in final_day_companies:
         if (x==j):
             companies1.append(x)
             break
len(companies1)

companies=companies1
# for i in df['SYMBOL'].unique()


In [172]:
j=1
ds=[]
for i in companies:
    ds.append(df[df['SYMBOL']==i])
index=0
final_companies=[]
final_prices=[]
print(ds,end="\n\n\n\n")
max_len=len(ds[0])
for i in ds:
    if (len(i)>max_len):
        max_len=len(i)
for i in ds: 
    if (len(i)==max_len):
        final_prices.append(i)
        final_companies.append(companies[index])
    index=index+1
len(final_prices)
print(len(final_companies))



63.85 2013-05-02
27641  WSTCSTPAPR     EQ   58.35 2013-06-03
86315  WSTCSTPAPR     EQ   54.70 2013-07-01
63441  WSTCSTPAPR     EQ   39.60 2013-08-01
87697  WSTCSTPAPR     EQ   39.00 2013-09-02
30580  WSTCSTPAPR     EQ   47.75 2013-10-01
21460  WSTCSTPAPR     EQ   47.70 2013-11-01
13706  WSTCSTPAPR     EQ   49.50 2013-12-02
83445  WSTCSTPAPR     EQ   50.25 2014-01-01
74120  WSTCSTPAPR     EQ   51.25 2014-03-03
62058  WSTCSTPAPR     EQ   54.25 2014-04-01
24614  WSTCSTPAPR     EQ   52.75 2014-05-02
81985  WSTCSTPAPR     EQ   57.45 2014-06-02
45396  WSTCSTPAPR     EQ   65.70 2014-07-01
16929  WSTCSTPAPR     EQ   56.00 2014-08-01
15334  WSTCSTPAPR     EQ   57.25 2014-09-01
69503  WSTCSTPAPR     EQ   58.55 2014-10-01
77226  WSTCSTPAPR     EQ   59.30 2014-11-03
67913  WSTCSTPAPR     EQ   58.60 2014-12-01
80378  WSTCSTPAPR     EQ   56.40 2015-01-01
72606  WSTCSTPAPR     EQ   59.90 2015-02-02
78801  WSTCSTPAPR     EQ   57.40 2015-03-02,           SYMBOL SERIES  CLOSE  TIMESTAMP
51668  XCHANGING

In [12]:
##### writing ocmpany wise prices to different sheets in excel ##########
# path='/Users/tejasvichabbra/Desktop/Portfolio_Analysis/out.xlsx'

# excel_book = pxl.load_workbook(path)
    
# for i in range(0,len(ds)):
#     with pd.ExcelWriter(path, engine='openpyxl') as writer: 
#         writer.book = excel_book
#         writer.sheets = {
#             worksheet.title: worksheet
#             for worksheet in excel_book.worksheets
#         }
#         ds[i].to_excel(writer, companies[i], index=False)
#         print(companies[i])
#         writer.save()
    


In [173]:
#calculating returns fro each comapny 
returns_list=[]
mean_list=[]
stddev_list=[]
for i in range(0,len(final_companies)): 
    prices=final_prices[i]['CLOSE']
    prices=prices.to_numpy()
    returns=np.zeros(len(prices)-1)
    j=-1
    for h in range((len(prices)-1),0,-1):
        j=j+1
        returns[j]=(prices[h]-prices[h-1])/prices[h-1]
        
    returns_list.append(returns)
    mean_list.append(returns.mean())
    stddev_list.append(returns.std())
len(mean_list)
len(returns_list)

415

In [174]:
len(stddev_list)

415

In [175]:
correlation=[]
i_index=0
j_index=0
corr_matrix=np.zeros(len(returns_list)**2).reshape(len(returns_list),len(returns_list))
cov_matrix=np.zeros(len(returns_list)**2).reshape(len(returns_list),len(returns_list))

for x in range(0,len(returns_list)):
    i=returns_list[x]
    for y in range(0,len(returns_list)):
        j=returns_list[y]
        corr_temp=np.corrcoef(i, j)
        corr=corr_temp[0][1]
        cov=corr*stddev_list[x]*stddev_list[y]
        corr_matrix[x][y]=corr
        cov_matrix[x][y]=cov
        

In [176]:
final_just_prices=[]
for i in final_prices:
    final_just_prices.append(i['CLOSE'])

In [ ]:
##### ALL THE VARIABLES CALCULATED TILL NOW ######

print(len(final_just_prices))# companywise prices arrays 
print(returns_list)# comapnywise returns arrays 
print(mean_list)# company wise mean returns 
print(stddev_list)# companywise stddev of returns
print(corr_matrix)# correlation matrix 
print(cov_matrix)#covariance matrix 

In [ ]:
##### forming a list of all the important variables #####
append_list=[]
append_list.append(final_just_prices)
append_list.append(returns_list)
append_list.append(mean_list)
append_list.append(stddev_list)
append_list.append(corr_matrix)
append_list.append(cov_matrix)
append_list.append(weight_list)

In [ ]:
##### writing all the important variables to different excel sheets of a workbook ###############
name_arrays=['companywise prices arrays','comapnywise returns arrays','company wise mean returns','companywise stddev of returns','correlation matrix','covariance matrix','weighatges']
path='/Users/tejasvichabbra/Desktop/Portfolio_Analysis/to_be_optimised_6yrs.xlsx'
excel_book = pxl.load_workbook(path)
def excel_writer(to_be_written,name):
    with pd.ExcelWriter(path, engine='openpyxl') as writer: 
        writer.book = excel_book
        writer.sheets = {
        worksheet.title: worksheet
        for worksheet in excel_book.worksheets
        }
        pd.DataFrame(to_be_written).to_excel(writer,name, index=False)
        writer.save()

excel_writer(weight_list,'weight_list')#weight 
excel_writer(mean_list,'mean_list')# company wise mean returns 
excel_writer(stddev_list,'stddev_list')# companywise stddev of returns
excel_writer(corr_matrix,'corr_matrix')# correlation matrix 
excel_writer(cov_matrix,'cov_matrix')#covariance matrix 

In [177]:
W = np.ones((len(returns_list),1))*(1.0/len(returns_list))
mean_list=np.array(mean_list)
def optimize(func, W, exp_ret, cov, target_return):
    opt_bounds = Bounds(0, 1)
    opt_constraints = ({'type': 'eq','fun': lambda W: 1.0 - np.sum(W)},{'type': 'eq','fun': lambda W: target_return - W.T@exp_ret})
    optimal_weights = minimize(func, W, 
                               args=(exp_ret, cov),
                               method='SLSQP',
                               bounds=opt_bounds,
                               constraints=opt_constraints)
    return optimal_weights['x']
def ret_risk(W, exp_ret, cov):
    return -((W.T@exp_ret) / (W.T@cov@W)**0.5)
x = optimize(ret_risk,W,mean_list, cov_matrix, 
             target_return=0.05)


In [178]:
expected_return=mean_list.T@x
expected_risk=(x.T@cov_matrix)@x
print(expected_return)
print(expected_risk)

0.05000000000005041
0.002043693336077596


In [180]:


tmp=final_day[final_day['SYMBOL']==final_companies[43]]
print(tmp,end='\n\n\n\n\n\n\n\n\n')
print(ctr,end='\n\n\n\n\n\n\n\n')
end=float(tmp['CLOSE'])

tmp1=first_day[first_day['SYMBOL']==final_companies[43]]
print(tmp1)
start=float(tmp1['CLOSE'])

         SYMBOL SERIES   OPEN   HIGH    LOW   CLOSE   LAST  PREVCLOSE  \
197  BALMLAWRIE     EQ  139.3  141.7  138.5  139.55  139.4     139.05   

     TOTTRDQTY   TOTTRDVAL    TIMESTAMP  TOTALTRADES          ISIN  \
197     351318  49180108.8  07-JUL-2021         4633  INE164A01016   

     Unnamed: 13  
197          NaN  








43







         SYMBOL SERIES    OPEN   HIGH    LOW   CLOSE   LAST  PREVCLOSE  \
141  BALMLAWRIE     EQ  568.05  580.9  562.2  576.75  580.0     560.65   

     TOTTRDQTY   TOTTRDVAL    TIMESTAMP  TOTALTRADES          ISIN  \
141      17423  9954523.45  01-APR-2015         1179  INE164A01016   

     Unnamed: 13  
141          NaN  


In [181]:
ctr=0
indi_final_returns=[]
path_first_day="/Users/tejasvichabbra/Desktop/Portfolio_Analysis/firstday_data1/firstday2.csv"
first_day=pd.read_csv(path_first_day)
first_day=first_day[first_day['SERIES']=='EQ']
print(first_day,end='\n\n\n\n\n')
for i in range(0,len(final_companies)):
    tmp=final_day[final_day['SYMBOL']==final_companies[i]]
    print(tmp,end='\n\n\n\n\n\n\n\n\n')
    print(ctr,end='\n\n\n\n\n\n\n\n')
    end=float(tmp['CLOSE'])
    tmp1=first_day[first_day['SYMBOL']==final_companies[i]]
    start=float(tmp1['CLOSE'])
    
    ctr+=1
   
    indi_final_returns.append(end-start)
individual_returns=np.array(indi_final_returns)
individual_returns=individual_returns.reshape(len(indi_final_returns),1)
x=x.reshape(len(x),1)
print(x.shape)
print(individual_returns.shape)
print(sum(x))
print(sum(individual_returns))
final_earnings=x.T@individual_returns
final_earnings
print(x)

LOSE   LAST  PREVCLOSE  \
1869  THOMASCOOK     EQ  64.65  66.1  63.9  64.75  64.65      64.75   

      TOTTRDQTY    TOTTRDVAL    TIMESTAMP  TOTALTRADES          ISIN  \
1869     674902  43952012.15  07-JUL-2021         5011  INE332A01027   

      Unnamed: 13  
1869          NaN  








379







         SYMBOL SERIES     OPEN      HIGH      LOW     CLOSE      LAST  \
1873  TIDEWATER     EQ  14734.0  15173.75  14000.0  15173.75  15173.75   

      PREVCLOSE  TOTTRDQTY    TOTTRDVAL    TIMESTAMP  TOTALTRADES  \
1873    14451.2      20760  307973447.7  07-JUL-2021         8807   

              ISIN  Unnamed: 13  
1873  INE484C01022          NaN  








380







          SYMBOL SERIES   OPEN  HIGH    LOW  CLOSE  LAST  PREVCLOSE  \
1879  TIMETECHNO     EQ  86.85  87.5  84.85  85.55  85.8      86.75   

      TOTTRDQTY   TOTTRDVAL    TIMESTAMP  TOTALTRADES          ISIN  \
1879     462668  39817285.9  07-JUL-2021         5481  INE508G01029   

      Unnamed: 13  
1879          NaN

In [138]:
col_names=['Expected Return' ,'Expected risk' ,'Final Earnings']
fin_df=pd.DataFrame(columns=col_names)
final_dic=[expected_return,expected_risk,float(final_earnings)]
a_series = pd.Series(final_dic, index = fin_df.columns)
fin_df = fin_df.append(a_series, ignore_index=True)
fin_df

NameError: name 'fin_day' is not defined

final_companies